The goal of this notebook is to implement CNN **Style Transfer**

In [2]:
%matplotlib inline

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms, models

In [ ]:
# Load VGG19 features

vgg = models.vgg19(pretrained=True).features

#By doing this, we freeze all VGG parameters since we're only optimizingg the target image
for param in vgg.parameters():
    param.require_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Emrah Sariboz/.cache\torch\checkpoints\vgg19-dcbb9e9d.pth
 44%|███████████████████████████████████                                             | 240M/548M [02:51<09:23, 573kB/s]

In [ ]:
device = torch.device("cuda" if torch.cuda_is_available() else "cpu")

vgg.to(device)

# Load in Content and Style Images